# MetPy calc

#### Provides tools for unit-aware, meteorological calculations in the following categories:

* Dry Thermodynamics
* Moist Thermodynamics
* Soundings
* Dynamic/Kinematic
* Boundary Layer/Turbulence
* Mathematical Functions
* Apparent Temperature
* Standard Atmosphere
* Other

Some methods require a vector or grid as input (e.g., calculations based on sounding profiles).

Those that do not can still be passed dimensional arrays, and will return like-sized arrays, even if they appear to be scalar functions.

In [1]:
import metpy.calc as calc
import metpy.constants as c
from metpy.units import units
import numpy as np

.


There are some very simple calculations, like potential temperature from pressure and temperature:

In [2]:
calc.potential_temperature(800. * units.hectopascal, 273. * units.kelvin)

290.9665329591883 <Unit('kelvin')>

### Series example
Given a list of model sigma coordinates in the vertical and the surface pressure, return the list of model pressure levels:

In [3]:
sigma = [0.995, 0.9821, 0.9644, 0.9425, 0.9159, 0.8838, 0.8458, 
         0.8014, 0.7508, 0.6943, 0.6329, 0.5681, 0.5017, 0.4357, 
         0.372, 0.3125, 0.2582, 0.2101, 0.1682, 0.1326, 0.1028, 
         0.0782, 0.058, 0.0418, 0.0288, 0.0183, 0.0101, 0.0027] * units.dimensionless
plevs = calc.sigma_to_pressure(sigma,1013.0 * units.millibar, 0.0 * units.millibar)
list(np.around(plevs,decimals=2))

[1007.94 <Unit('millibar')>,
 994.87 <Unit('millibar')>,
 976.94 <Unit('millibar')>,
 954.75 <Unit('millibar')>,
 927.81 <Unit('millibar')>,
 895.29 <Unit('millibar')>,
 856.8 <Unit('millibar')>,
 811.82 <Unit('millibar')>,
 760.56 <Unit('millibar')>,
 703.33 <Unit('millibar')>,
 641.13 <Unit('millibar')>,
 575.49 <Unit('millibar')>,
 508.22 <Unit('millibar')>,
 441.36 <Unit('millibar')>,
 376.84 <Unit('millibar')>,
 316.56 <Unit('millibar')>,
 261.56 <Unit('millibar')>,
 212.83 <Unit('millibar')>,
 170.39 <Unit('millibar')>,
 134.32 <Unit('millibar')>,
 104.14 <Unit('millibar')>,
 79.22 <Unit('millibar')>,
 58.75 <Unit('millibar')>,
 42.34 <Unit('millibar')>,
 29.17 <Unit('millibar')>,
 18.54 <Unit('millibar')>,
 10.23 <Unit('millibar')>,
 2.74 <Unit('millibar')>]

In [4]:
plevs.to(units.pascal)

array([100793.5 ,  99486.73,  97693.72,  95475.25,  92780.67,  89528.94,
        85679.54,  81181.82,  76056.04,  70332.59,  64112.77,  57548.53,
        50822.21,  44136.41,  37683.6 ,  31656.25,  26155.66,  21283.13,
        17038.66,  13432.38,  10413.64,   7921.66,   5875.4 ,   4234.34,
         2917.44,   1853.79,   1023.13,    273.51]) <Unit('pascal')>

### Nesting
Functions can be nested.
For example, there is not a function to calculate specific humidity directly from relative humidity, but there are functions for relative humidity to dewpoint, and then dewpoint to specific humidity:

In [5]:
calc.specific_humidity_from_dewpoint(calc.dewpoint_rh(95 * units.degF, 50 * units.percent), 29.92 * units.inHg)

/homes/kpegion/.conda/envs/aoes/lib/python3.6/site-packages/metpy/xarray.py:655: MetpyDeprecationWarning: The dewpoint_rh function was deprecated in version 0.12. This function has been renamed dewpoint_from_relative_humidity.
  return func(*args, **kwargs)


0.01746755638387136 <Unit('dimensionless')>

### More powerful functions
Here is an example of calculating CAPE and CIN from profiles of temperature and dewpoint (e.g., from a radiosonde):

In [6]:
t_profile = [299.10, 300.08, 300.18, 300.28, 300.40, 300.50, 300.57, 300.59,
       300.56, 300.52, 300.59, 300.26, 299.30, 298.96, 298.47, 297.90,
       297.23, 296.46, 295.58, 294.68, 293.94, 293.46, 293.13, 292.65,
       291.96, 291.15, 290.26, 289.34, 288.23, 286.91, 285.51, 284.15,
       281.99, 280.00, 278.20, 276.52, 275.46, 273.96, 271.22, 269.24,
       265.27, 265.24, 264.11, 262.99, 260.92, 259.68, 257.17, 256.84,
       254.82, 252.79, 250.66, 248.49, 246.34, 244.41, 242.85, 240.85] * units.kelvin
td_profile = [294.94, 295.25, 295.18, 295.12, 295.07, 295.01, 294.95, 294.88,
       294.79, 294.66, 294.41, 293.35, 292.61, 292.12, 291.72, 291.28,
       290.77, 290.63, 290.54, 290.38, 289.62, 287.89, 285.89, 284.3 ,
       282.83, 281.19, 279.36, 277.59, 276.23, 275.34, 273.88, 270.3 ,
       263.84, 256.68, 250.94, 247.86, 243.36, 240.34, 237.07, 239.96,
       238.58, 235.17, 232.41, 231.57, 233.72, 240.21, 244.34, 243.2 ,
       239.54, 236.16, 234.42, 235.03, 237.08, 235.84, 230.25, 224.31] * units.kelvin
p_profile = [978.08, 976.92, 974.49, 971.84, 968.94, 965.78, 962.34, 958.58,
       954.49, 950.03, 945.19, 939.94, 934.23, 928.06, 921.37, 914.15,
       906.36, 897.97, 888.95, 879.27, 868.92, 857.86, 846.07, 833.54,
       820.27, 806.24, 791.47, 775.96, 759.73, 742.82, 725.26, 707.1 ,
       688.41, 669.24, 649.67, 629.79, 609.69, 589.47, 569.21, 549.02,
       529.01, 509.26, 489.88, 471.01, 452.69, 434.94, 417.76, 401.11,
       385.01, 369.42, 354.35, 339.77, 325.68, 312.07, 298.92, 286.22] * units.hectopascal
calc.surface_based_cape_cin(p_profile, t_profile, td_profile)

(1120.4139829861845 <Unit('joule / kilogram')>,
 -188.78510536438793 <Unit('joule / kilogram')>)

### Dry Thermodynamics
`add_height_to_pressure(pressure, height)` Calculate the pressure at a certain height above another pressure level.

`add_pressure_to_height(height, pressure)` Calculate the height at a certain pressure above another height.

`density(pressure, temperature, mixing[, …])` Calculate density.

`dry_lapse(pressure, temperature[, ref_pressure])` Calculate the temperature at a level assuming only dry processes.

`dry_static_energy(heights, temperature)` Calculate the dry static energy of parcels.

`geopotential_to_height(geopot)` Compute height from a given geopotential.

`height_to_geopotential(height)` Compute geopotential for a given height.

`mean_pressure_weighted(pressure, *args, **kwargs)` Calculate pressure-weighted mean of an arbitrary variable through a layer.

`potential_temperature(pressure, temperature)` Calculate the potential temperature.

`sigma_to_pressure(sigma, psfc, ptop)` Calculate pressure from sigma values.

`static_stability(pressure, temperature[, axis])` Calculate the static stability within a vertical profile.

`temperature_from_potential_temperature(…)` Calculate the temperature from a given potential temperature.

`thickness_hydrostatic(pressure, temperature, …)` Calculate the thickness of a layer via the hypsometric equation.

### Moist Thermodynamics
`dewpoint(e)` Calculate the ambient dewpoint given the vapor pressure.

`dewpoint_from_specific_humidity(…)` Calculate the dewpoint from specific humidity, temperature, and pressure.

`dewpoint_rh(temperature, rh)` Calculate the ambient dewpoint given air temperature and relative humidity.

`equivalent_potential_temperature(pressure, …)` Calculate equivalent potential temperature.

`mixing_ratio(part_press, tot_press[, …])` Calculate the mixing ratio of a gas.

`mixing_ratio_from_relative_humidity(…)` Calculate the mixing ratio from relative humidity, temperature, and pressure.

`mixing_ratio_from_specific_humidity(…)` Calculate the mixing ratio from specific humidity.

`moist_lapse(pressure, temperature[, …])` Calculate the temperature at a level assuming liquid saturation processes.

`moist_static_energy(heights, temperature, …)` Calculate the moist static energy of parcels.

`precipitable_water(dewpt, pressure[, …])` Calculate precipitable water through the depth of a sounding.

`psychrometric_vapor_pressure_wet(…[, …])` Calculate the vapor pressure with wet bulb and dry bulb temperatures.

`relative_humidity_from_dewpoint(temperature, …)` Calculate the relative humidity.

`relative_humidity_from_mixing_ratio(…)` Calculate the relative humidity from mixing ratio, temperature, and pressure.

`relative_humidity_from_specific_humidity(…)` Calculate the relative humidity from specific humidity, temperature, and pressure.

`relative_humidity_wet_psychrometric(…)` Calculate the relative humidity with wet bulb and dry bulb temperatures.

`saturation_equivalent_potential_temperature(…)` Calculate saturation equivalent potential temperature.

`saturation_mixing_ratio(tot_press, temperature)` Calculate the saturation mixing ratio of water vapor.

`saturation_vapor_pressure(temperature)` Calculate the saturation water vapor (partial) pressure.

`specific_humidity_from_dewpoint(dewpoint, …)` Calculate the specific humidity from the dewpoint temperature and pressure.

`specific_humidity_from_mixing_ratio(mixing_ratio)` Calculate the specific humidity from the mixing ratio.

`thickness_hydrostatic_from_relative_humidity(…)` Calculate the thickness of a layer given pressure, temperature and relative humidity.

`vapor_pressure(pressure, mixing)` Calculate water vapor (partial) pressure.

`vertical_velocity(omega, pressure, temperature)` Calculate w from omega assuming hydrostatic conditions.

`vertical_velocity_pressure(w, pressure, …)` Calculate omega from w assuming hydrostatic conditions.

`virtual_potential_temperature(pressure, …)` Calculate virtual potential temperature.

`virtual_temperature(temperature, mixing[, …])` Calculate virtual temperature.

`wet_bulb_temperature(pressure, temperature, …)` Calculate the wet-bulb temperature using Normand’s rule.

### Soundings
`bulk_shear(pressure, u, v[, heights, …])` Calculate bulk shear through a layer.

`bunkers_storm_motion(pressure, u, v, heights)` 
Calculate the Bunkers right-mover and left-mover storm motions and sfc-6km mean flow.

`cape_cin(pressure, temperature, dewpt, …)` Calculate CAPE and CIN.

`critical_angle(pressure, u, v, heights, …)` Calculate the critical angle.

`el(pressure, temperature, dewpt[, …])` Calculate the equilibrium level.

`lcl(pressure, temperature, dewpt[, …])` Calculate the lifted condensation level (LCL) using from the starting point.

`lfc(pressure, temperature, dewpt[, …])` Calculate the level of free convection (LFC).

`mixed_layer(p, *args, **kwargs)` Mix variable(s) over a layer, yielding a mass-weighted average.

`mixed_parcel(p, temperature, dewpt[, …])` Calculate the properties of a parcel mixed from a layer.

`most_unstable_cape_cin(pressure, …)` Calculate most unstable CAPE/CIN.

`most_unstable_parcel(pressure, temperature, …)` Determine the most unstable parcel in a layer.

`parcel_profile(pressure, temperature, dewpt)` Calculate the profile a parcel takes through the atmosphere.

`parcel_profile_with_lcl(pressure, …)` Calculate the profile a parcel takes through the atmosphere.

`significant_tornado(sbcape, …)` Calculate the significant tornado parameter (fixed layer).

`storm_relative_helicity(u, v, heights, depth)` Calculate storm relative helicity.

`supercell_composite(mucape, …)` Calculate the supercell composite parameter.

`surface_based_cape_cin(pressure, …)` Calculate surface-based CAPE and CIN.

### Dynamic/Kinematic
`absolute_momentum(u_wind, v_wind[, index])` Calculate cross-sectional absolute momentum (also called pseudoangular momentum).

`absolute_vorticity(u, v, dx, dy, lats[, …])` Calculate the absolute vorticity of the horizontal wind.

`advection(scalar, wind, deltas)` Calculate the advection of a scalar field by the wind.

`ageostrophic_wind(heights, f, dx, dy, u, v)` Calculate the ageostrophic wind given from the heights or geopotential.

`coriolis_parameter(latitude)` Calculate the coriolis parameter at each point.

`divergence(u, v, dx, dy)` Calculate the horizontal divergence of a vector.

`exner_function(pressure[, reference_pressure])` Calculate the Exner function.

`frontogenesis(thta, u, v, dx, dy[, dim_order])` Calculate the 2D kinematic frontogenesis of a temperature field.

`geostrophic_wind(heights, f, dx, dy)` Calculate the geostrophic wind given from the heights or geopotential.

`inertial_advective_wind(u, v, u_geostrophic, …)` Calculate the inertial advective wind.

`kinematic_flux(vel, b[, perturbation, axis])` Compute the kinematic flux from two time series.

`montgomery_streamfunction(height, temperature)` Compute the Montgomery Streamfunction on isentropic surfaces.

`potential_vorticity_baroclinic(…)` Calculate the baroclinic potential vorticity.

`potential_vorticity_barotropic(heights, u, …)` Calculate the barotropic (Rossby) potential vorticity.

`q_vector(u, v, temperature, pressure, dx, dy)` Calculate Q-vector at a given pressure level using the u, v winds and temperature.

`shearing_deformation(u, v, dx, dy)` Calculate the shearing deformation of the horizontal wind.

`stretching_deformation(u, v, dx, dy)` Calculate the stretching deformation of the horizontal wind.

`total_deformation(u, v, dx, dy)` Calculate the horizontal total deformation of the horizontal wind.

`vorticity(u, v, dx, dy)` Calculate the vertical vorticity of the horizontal wind.

`wind_components(speed, wdir)` Calculate the U, V wind vector components from the speed and direction.

`wind_direction(u, v[, convention])` Compute the wind direction from u and v-components.

`wind_speed(u, v)` Compute the wind speed from u and v-components.

### Boundary Layer/Turbulence
`brunt_vaisala_frequency(heights, …[, axis])` Calculate the Brunt-Vaisala frequency.

`brunt_vaisala_frequency_squared(heights, …)` Calculate the square of the Brunt-Vaisala frequency.

`brunt_vaisala_period(heights, …[, axis])` Calculate the Brunt-Vaisala period.

`friction_velocity(u, w[, v, perturbation, axis])` Compute the friction velocity from the time series of velocity components.

`tke(u, v, w[, perturbation, axis])` Compute turbulence kinetic energy.

### Mathematical Functions
`cross_section_components(data_x, data_y[, index])` Obtain the tangential and normal components of a cross-section of a vector field.

`first_derivative(f, **kwargs)` Calculate the first derivative of a grid of values.

`gradient(f, **kwargs)` Calculate the gradient of a grid of values.

`grid_deltas_from_dataarray(f)` Calculate the horizontal deltas between grid points of a DataArray.

`laplacian(f, **kwargs)` Calculate the laplacian of a grid of values.

`lat_lon_grid_deltas(longitude, latitude, …)` Calculate the delta between grid points that are in a latitude/longitude format.

`normal_component(data_x, data_y[, index])` Obtain the normal component of a cross-section of a vector field.

`second_derivative(f, **kwargs)` Calculate the second derivative of a grid of values.

`tangential_component(data_x, data_y[, index])` Obtain the tangential component of a cross-section of a vector field.

`unit_vectors_from_cross_section(cross[, index])` Calculate the unit tanget and unit normal vectors from a cross-section.

### Apparent Temperature
`apparent_temperature(temperature, rh, speed)` Calculate the current apparent temperature.

`heat_index(temperature, rh[, mask_undefined])` Calculate the Heat Index from the current temperature and relative humidity.

`windchill(temperature, speed[, …])` Calculate the Wind Chill Temperature Index (WCTI).

### Standard Atmosphere
`altimeter_to_sea_level_pressure(…)` Convert the altimeter setting to sea-level pressure.

`altimeter_to_station_pressure(…)` Convert the altimeter measurement to station pressure.

`height_to_pressure_std(height)` Convert height data to pressures using the U.S.

`pressure_to_height_std(pressure)` Convert pressure data to heights using the U.S.

### Other
`angle_to_direction(input_angle[, full, level])` Convert the meteorological angle to directional text.

`find_bounding_indices(arr, values, axis[, …])` Find the indices surrounding the values within arr along axis.

`find_intersections(x, a, b[, direction, log_x])` Calculate the best estimate of intersection.

`get_layer(pressure, *args, **kwargs)` Return an atmospheric layer from upper air data with the requested bottom and depth.

`get_layer_heights(heights, depth, *args, …)` Return an atmospheric layer from upper air data with the requested bottom and depth.

`get_perturbation(ts[, axis])` Compute the perturbation from the mean of a time series.

`isentropic_interpolation(theta_levels, …)` Interpolate data in isobaric coordinates to isentropic coordinates.

`nearest_intersection_idx(a, b)` Determine the index of the point just before two lines with common x values.

`parse_angle(input_dir)` Calculate the meteorological angle from directional text.

`reduce_point_density(points, radius[, priority])` Return a mask to reduce the density of points in irregularly-spaced data.

`resample_nn_1d(a, centers)` Return one-dimensional nearest-neighbor indexes based on user-specified centers.

`smooth_gaussian(scalar_grid, n)` Filter with normal distribution of weights.

`smooth_n_point(scalar_grid[, n, passes])` Filter with normal distribution of weights.